# <center>Создание адаптера для класса

Вы продолжаете писать игру, и настало время разобраться с расчетом освещенности на карте. Так как это не совсем тривиальная задача, вы хотели бы использовать готовое решение, а не писать свое собственное. Вам удалось найти готовый класс, который решает задачу, однако интерфейс этого класса не совместим с вашей игрой.

Вам нужно написать адаптер, который позволил бы использовать найденный вами класс совместно с вашей системой.

Интерфейс класса выглядит следующим образом:

In [1]:
from abc import ABC, abstractmethod

In [2]:
class Light:
    def __init__(self, dim):
        self.dim = dim
        self.grid = [[0 for i in range(dim[0])] for _ in range(dim[1])]
        self.lights = []
        self.obstacles = []
        
    def set_dim(self, dim):
        self.dim = dim
        self.grid = [[0 for i in range(dim[0])] for _ in range(dim[1])]
    
    def set_lights(self, lights):
        self.lights = lights
        self.generate_lights()
    
    def set_obstacles(self, obstacles):
        self.obstacles = obstacles
        self.generate_lights()
        
    def generate_lights(self):
        return self.grid.copy()


Интерфейс системы выглядит следующим образом:

In [3]:
class System:
    def __init__(self):
        self.map = self.grid = [[0 for i in range(30)] for _ in range(20)]
        self.map[5][7] = 1 # Источники света
        self.map[5][2] = -1 # Стены
    
    def get_lightening(self, light_mapper):
        self.lightmap = light_mapper.lighten(self.map)


In [4]:
light = Light((10, 7))

In [5]:
light.set_lights([(6, 4), (2, 1)])

In [6]:
light.generate_lights()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Класс Light создает в методе __init__ поле заданного размера. За размер поля отвечает параметр, представляющий из себя кортеж из 2 чисел. Элемент dim[1] отвечает за высоту карты, dim[0] за ее ширину. Метод set_lights устанавливает массив источников света с заданными координатами и просчитывает освещение. Метод set_obstacles устанавливает препятствия аналогичным образом. Положение элементов задается списком кортежей. В каждом элементе кортежа хранятся 2 значения: elem[0] -- координата по ширине карты и elem[1] -- координата по высоте соответственно. Метод generate_lights рассчитывает освещенность с учетом источников и препятствий.

Пример карты освещенности, полученной этим методом изображен на следующем рисунке:

<img src="images/light.jpg">

В системе в конструкторе создается двухмерная, карта, на которой источники света обозначены как 1, а препятствия как -1. Метод get_lightening принимает в качестве аргумента объект, который должен посчитывать освещение. У объекта вызывается метод lighten, который принимает карту объектов и источников света и возвращает карту освещенности.

Вам необходимо написать адаптер MappingAdapter. Прототип класса вам дан в качестве исходного кода.

In [7]:
class MappingAdapterAbstract(ABC):  # sets the interface
    def __init__(self, adaptee):
        pass
    
    @abstractmethod
    def lighten(self, grid):
        pass

In [25]:
class MappingAdapter(Light, MappingAdapterAbstract):
    def __init__(self, adaptee):
        self.adaptee = adaptee
    
    def _get_lights_and_obstacles(self, grid):
        lights = []
        obstacles = []
        for i, row in enumerate(grid):
            for j, elem in enumerate(row):
                if elem == 1:
                    lights += [(j, i)]
                if elem == -1:
                    obstacles += [(j, i)]
        return lights, obstacles
    
    def lighten(self, grid):
        dim = (len(grid[0]), len(grid))
        self.adaptee.set_dim(dim)
        lights, obstacles = self._get_lights_and_obstacles(grid)
        self.adaptee.set_lights(lights)
        self.adaptee.set_obstacles(obstacles)
        return self.adaptee.generate_lights()

In [26]:
dim = (10, 20)

system = System()
light_mapper = Light(dim)
adapter = MappingAdapter(light_mapper)
system.get_lightening(adapter)